In [1]:
from sunpy.net import Fido
from sunpy.net import attrs as a
import pandas as pd

/home/griffingoodwin/anaconda3/envs/down/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
goes_query = Fido.search(
    a.Time("2020-09-11", "2020-10-30"),
    a.Instrument('XRS'),
    a.Resolution('avg1m'))

In [3]:
goes_query

Start Time,End Time,Instrument,Physobs,Source,Provider,Resolution,SatelliteNumber
Time,Time,str3,str10,str4,str4,str5,int64
2020-09-11 00:00:00.000,2020-09-11 23:59:59.999,XRS,irradiance,GOES,NOAA,avg1m,16
2020-09-12 00:00:00.000,2020-09-12 23:59:59.999,XRS,irradiance,GOES,NOAA,avg1m,16
2020-09-13 00:00:00.000,2020-09-13 23:59:59.999,XRS,irradiance,GOES,NOAA,avg1m,16
2020-09-14 00:00:00.000,2020-09-14 23:59:59.999,XRS,irradiance,GOES,NOAA,avg1m,16
2020-09-15 00:00:00.000,2020-09-15 23:59:59.999,XRS,irradiance,GOES,NOAA,avg1m,16
2020-09-16 00:00:00.000,2020-09-16 23:59:59.999,XRS,irradiance,GOES,NOAA,avg1m,16
2020-09-17 00:00:00.000,2020-09-17 23:59:59.999,XRS,irradiance,GOES,NOAA,avg1m,16
2020-09-18 00:00:00.000,2020-09-18 23:59:59.999,XRS,irradiance,GOES,NOAA,avg1m,16
2020-09-19 00:00:00.000,2020-09-19 23:59:59.999,XRS,irradiance,GOES,NOAA,avg1m,16


In [3]:
event_type = "FL"
tstart = "2010/01/01"
tend = "2025/7/01"
result = Fido.search(a.Time(tstart, tend),
                     a.hek.EventType(event_type),
                     a.hek.FL.GOESCls >= "M1.0",
                     a.hek.OBS.Observatory == "GOES")

In [4]:
hek_results = result["hek"]

In [5]:
print(hek_results['hgc_coord'])

      hgc_coord      
---------------------
   POINT(148.69204 0)
    POINT(144.6688 0)
  POINT(51.98143 -24)
  POINT(50.46024 -24)
   POINT(136.77781 0)
  POINT(51.90091 -26)
 POINT(-108.44932 21)
   POINT(-92.94885 0)
 POINT(-105.59132 21)
  POINT(-111.64737 0)
                  ...
   POINT(60.395845 0)
POINT(122.487279 -14)
     POINT(53.9522 0)
    POINT(49.97202 0)
  POINT(34.245325 19)
   POINT(41.404952 0)
   POINT(40.209971 0)
   POINT(38.408308 0)
    POINT(21.35678 0)
   POINT(32.55938 19)
   POINT(-15.99142 0)
Length = 2504 rows


In [6]:
filtered_results = hek_results["event_starttime", "event_peaktime",
                               "event_endtime", "fl_goescls", "ar_noaanum", "hgc_coord"]

In [7]:
flare_df = filtered_results.to_pandas()
flare_df

,event_starttime,event_peaktime,event_endtime,fl_goescls,ar_noaanum,hgc_coord
0,2010-01-19 13:03:00,2010-01-19 13:41:00,2010-01-19 13:50:00,M2.3,11041,POINT(148.69204 0)
1,2010-01-19 20:23:00,2010-01-19 20:35:00,2010-01-19 20:46:00,M1.7,11041,POINT(144.6688 0)
2,2010-01-20 06:45:00,2010-01-20 07:27:00,2010-01-20 07:34:00,M1.0,11041,POINT(51.98143 -24)
3,2010-01-20 07:42:00,2010-01-20 07:49:00,2010-01-20 07:53:00,M1.6,11041,POINT(50.46024 -24)
4,2010-01-20 10:46:00,2010-01-20 10:59:00,2010-01-20 11:10:00,M1.8,11041,POINT(136.77781 0)
...,...,...,...,...,...,...
2499,2025-06-16 11:27:00,2025-06-16 11:39:00,2025-06-16 11:43:00,M1.4,0,POINT(40.209971 0)
2500,2025-06-16 14:43:00,2025-06-16 14:55:00,2025-06-16 15:03:00,M1.1,0,POINT(38.408308 0)
2501,2025-06-17 21:38:00,2025-06-17 21:49:00,2025-06-17 21:54:00,X1.2,14114,POINT(21.35678 0)
2502,2025-06-20 00:02:00,2025-06-20 00:04:00,2025-06-20 00:06:00,M4.6,14114,POINT(32.55938 19)


In [10]:
flare_df[(flare_df['event_starttime'] < '2023-08-15') & (flare_df['event_starttime'] > '2023-07-01')]


,event_starttime,event_peaktime,event_endtime,fl_goescls,ar_noaanum,hgc_coord
1240,2023-07-01 22:07:00,2023-07-01 22:23:00,2023-07-01 22:31:00,M1.1,0,POINT(124.1085 0)
1241,2023-07-02 02:29:00,2023-07-02 02:35:00,2023-07-02 02:40:00,M2.0,0,POINT(121.70026 0)
1242,2023-07-03 06:43:00,2023-07-03 06:53:00,2023-07-03 06:59:00,M1.3,0,POINT(106.12955 0)
1243,2023-07-04 12:20:00,2023-07-04 12:35:00,2023-07-04 12:55:00,M1.4,0,POINT(89.796184 0)
1244,2023-07-06 10:32:00,2023-07-06 10:49:00,2023-07-06 11:07:00,M1.0,0,POINT(64.317823 0)
...,...,...,...,...,...,...
1298,2023-08-07 04:29:00,2023-08-07 04:41:00,2023-08-07 04:51:00,M2.4,0,POINT(4.2957537 0)
1299,2023-08-07 15:30:00,2023-08-07 16:27:00,2023-08-07 16:36:00,M1.0,0,POINT(-1.77388999999999 0)
1300,2023-08-07 19:37:00,2023-08-07 19:51:00,2023-08-07 19:58:00,M1.4,0,POINT(-4.04194000000001 0)
1301,2023-08-07 20:30:00,2023-08-07 20:46:00,2023-08-07 21:18:00,X1.5,0,POINT(-4.52859999999998 0)


In [22]:
# Summarize flare events
print(f"Total number of flares: {len(flare_df)}")

# Count flares by GOES class
goes_class_counts = flare_df['fl_goescls'].str[0].value_counts()
print("\nFlares by GOES class:")
print(goes_class_counts)

# Get time span
time_span = pd.to_datetime(flare_df['event_starttime'])
print(f"\nTime span: {time_span.min()} to {time_span.max()}")

# Count flares per year
flares_by_year = flare_df['event_starttime'].dt.year.value_counts().sort_index()
print("\nFlares per year:")
print(flares_by_year)

# Show most active regions
active_regions = flare_df['ar_noaanum'].value_counts().head()
print("\nMost active regions (top 5):")
print(active_regions)


Total number of flares: 2504

Flares by GOES class:
fl_goescls
M    2366
X     137
S       1
Name: count, dtype: int64

Time span: 2010-01-19 13:03:00 to 2025-06-20 17:21:00

Flares per year:
event_starttime
2010     24
2011    120
2012    136
2013    111
2014    226
2015    132
2016     16
2017     43
2020      2
2021     31
2022    196
2023    334
2024    950
2025    183
Name: count, dtype: int64

Most active regions (top 5):
ar_noaanum
0        588
13664     64
13615     42
13663     40
12192     38
Name: count, dtype: int64


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load GOES data
goes_combined = pd.read_csv('/home/griffingoodwin/downloads/goes_combined/combined_g18_avg1m_20230701_20230815.csv')
goes_combined['time'] = pd.to_datetime(goes_combined['time'], errors='coerce')

# Load flare catalog
#flare_df = pd.read_csv('/path/to/flare_catalog.csv')  # <-- Replace with actual path
flare_df['event_starttime'] = pd.to_datetime(flare_df['event_starttime'], errors='coerce')
flare_df['event_peaktime'] = pd.to_datetime(flare_df['event_peaktime'], errors='coerce')
flare_df['event_endtime'] = pd.to_datetime(flare_df['event_endtime'], errors='coerce')

# Plot GOES flux
plt.figure(figsize=(14, 6))
plt.plot(goes_combined['time'], goes_combined['xrsb_flux'], label='GOES XRSB Flux', color='blue')

# Overlay flares

for _, flare in flare_df.iterrows():
    start = flare['event_starttime']
    peak = flare['event_peaktime']
    end = flare['event_endtime']

    if peak > pd.Timestamp("2023-08-15") or peak < pd.Timestamp("2023-07-01"):
        continue

    if pd.notna(start) and pd.notna(end) and start < end:
        plt.axvspan(start.to_pydatetime(), end.to_pydatetime(), color='red', alpha=0.2)

    if pd.notna(peak):
        plt.axvline(peak.to_pydatetime(), color='red', linestyle='--', alpha=0.5)

# Final plot settings
plt.yscale('log')
plt.xlabel('Time')
plt.ylabel('Flux [W/m²]')
plt.title('GOES XRSB Flux with Flare Events (Shaded & Peak Lines)')
plt.legend()
plt.tight_layout()
plt.show()
